## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iqaluit,CA,63.75,-68.51,48.20,76,40,3.36,scattered clouds
1,1,Sarahan,IN,31.52,77.80,63.54,76,54,1.45,broken clouds
2,2,Puerto Ayora,EC,-0.74,-90.35,75.20,60,40,14.99,scattered clouds
3,3,Saskylakh,RU,71.92,114.08,40.93,83,0,8.16,clear sky
4,4,Port Hedland,AU,-20.32,118.57,66.20,55,0,8.05,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 40


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]

In [7]:
# 4a. Determine if there are any empty rows.
filtered_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 7 to 685
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              26 non-null     int64  
 1   City                 26 non-null     object 
 2   Country              26 non-null     object 
 3   Lat                  26 non-null     float64
 4   Lng                  26 non-null     float64
 5   Max Temp             26 non-null     float64
 6   Humidity             26 non-null     int64  
 7   Cloudiness           26 non-null     int64  
 8   Wind Speed           26 non-null     float64
 9   Current Description  26 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 2.2+ KB


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities.dropna()
clean_df.shape

(26, 10)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
395,Belaya Gora,RU,34.07,overcast clouds,68.53,146.42,
401,Seymchan,RU,33.46,overcast clouds,62.88,152.43,
413,Arman,RU,39.51,clear sky,59.70,150.17,
486,Canberra,AU,35.60,clear sky,-35.28,149.13,
527,Bilibino,RU,38.53,overcast clouds,68.05,166.44,
605,Swan Hill,AU,35.55,clear sky,-35.34,143.55,
618,Coihaique,CL,39.20,light rain,-45.58,-72.07,
619,Deputatskiy,RU,30.63,overcast clouds,69.30,139.90,
638,Ust-Omchug,RU,28.96,clear sky,61.15,149.63,
685,Esso,RU,36.59,broken clouds,55.93,158.70,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

7
8
13
37
74
124
Hotel not found... skipping.
145
Hotel not found... skipping.
176
198
203
204
246
247
Hotel not found... skipping.
253
257
281
Hotel not found... skipping.
395
Hotel not found... skipping.
401
Hotel not found... skipping.
413
Hotel not found... skipping.
486
527
605
618
619
638
Hotel not found... skipping.
685


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """ 
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))